# Coursera - Finding Hidden Messages on DNA

In [1]:
def Solve(Answer):
    return " ".join(list(map(str, Answer)))

To compute Count(Text, Pattern), our plan is to “slide a window” down Text, checking whether each k-mer substring of Text matches Pattern. We will therefore refer to the k-mer starting at position i of Text as Text(i, k). Throughout this book, we will often use 0-based indexing, meaning that we count starting at 0 instead of 1. In this case, Text begins at position 0 and ends at position |Text| − 1 (|Text| denotes the number of symbols in Text). For example, if Text = GACCATACTG, then Text(4, 3) = ATA. Note that the last k-mer of Text begins at position |Text| − k, e.g., the last 3-mer of GACCATACTG starts at position 10 − 3 = 7. This discussion results in the following pseudocode for computing Count(Text, Pattern).

```
PatternCount(Text, Pattern)
    count ← 0
    for i ← 0 to |Text| − |Pattern|
        if Text(i, |Pattern|) = Pattern
            count ← count + 1
    return count
```

In [2]:
def PatternCount(Text, Pattern):
    count = 0
    l = len(Pattern)
    for i in range((len(Text) + 1) - l):
        if Text[i:i+l] == Pattern:
            count += 1
    return count

In [3]:
genome, pattern = 'GCGCG', 'GCG'

PatternCount(genome, pattern)

2

A straightforward algorithm for finding the most frequent k-mers in a string Text checks all k-mers appearing in this string (there are |Text| − k + 1 such k-mers) and then computes how many times each k-mer appears in Text. To implement this algorithm, called FrequentWords, we will need to generate an array Count, where Count(i) stores Count(Text, Pattern) for Pattern = Text(i, k) (see figure below).

<img src="https://ucarecdn.com/8367f24c-c989-4ad1-b5a4-9ab2dafa3a10/">

Figure: The array Count for Text = ACTGACTCCCACCCC and k = 3. For example, Count(0) = Count(4) = 2 because ACT (shown in boldface) appears twice in Text.

The pseudocode for FrequentWords is shown below.

```
FrequentWords(Text, k)
    FrequentPatterns ← an empty set
    for i ← 0 to |Text| − k
        Pattern ← the k-mer Text(i, k)
        Count(i) ← PatternCount(Text, Pattern)
    maxCount ← maximum value in array Count
    for i ← 0 to |Text| − k
        if Count(i) = maxCount
            add Text(i, k) to FrequentPatterns
    remove duplicates from FrequentPatterns
    return FrequentPatterns
```

In [4]:
def FrequentWords(Text, k):
    FrequentPatterns = []
    l = len(Text)
    Count = {}
    for i in range((l + 1) - k):
        Pattern = Text[i:i+k]
        Count[Pattern] = PatternCount(Text, Pattern)
    maxCount = max(Count.values())
    for pattern, count in Count.items():
        if count == maxCount:
            FrequentPatterns.append(pattern)
    return set(FrequentPatterns)

In [5]:
genome, k = 'ACGTTGCATGTCGCATGATGCATGAGAGCT', 4

FrequentWords(genome, k)

{'CATG', 'GCAT'}

Recall that nucleotides A and T are complements of each other, as are G and C. Having one strand and a supply of “free floating” nucleotides, one can imagine the synthesis of a complementary strand on a template strand. This model of replication was confirmed rigorously by Meselson and Stahl in 1958. The beginning and end of a DNA strand are denoted 5’ (pronounced “five prime”) and 3’ (pronounced “three prime”), respectively. For more details, see "DETOUR: The Most Beautiful Experiment in Biology" in the print companion.

The figure below shows a template strand AGTCGCATAGT and its complementary strand ACTATGCGACT.

<img src="http://bioinformaticsalgorithms.com/images/Replication/reverse_complement.png">

At this point, you may think that we have made a mistake, since the complementary strand in this figure reads out TCAGCGTATCA from left to right rather than ACTATGCGACT. We have not: each DNA strand has a direction, and the complementary strand runs in the opposite direction to the template strand, as shown by the arrows in the figure.

Given a nucleotide p, we denote its complementary nucleotide as p*. The reverse complement of a string Pattern = p1 … pn is the string Patternrc = pn* … p1* formed by taking the complement of each nucleotide in Pattern, then reversing the resulting string. We will need the solution to the following problem throughout this chapter:

```
Reverse Complement Problem: Find the reverse complement of a DNA string.
     Input: A DNA string Pattern.
     Output: Patternrc , the reverse complement of Pattern.
```

In [6]:
def ReverseComplement(Pattern):
    Pattern_rc = []
    for nucleotide in Pattern:
        if nucleotide == 'G':
            Pattern_rc.append("C")
        elif nucleotide == 'C':
            Pattern_rc.append("G")
        elif nucleotide == 'A':
            Pattern_rc.append("T")
        else:
            Pattern_rc.append("A")
    Pattern_rc.reverse()
    return "".join(Pattern_rc)

In [7]:
genome = 'AAAACCCGGT'

ReverseComplement(genome)

'ACCGGGTTTT'

Interestingly, among the four most frequent 9-mers in ori of Vibrio cholerae, ATGATCAAG and CTTGATCAT are reverse complements of each other, resulting in the six total occurrences of these strings shown below.


`atcaatgatcaacgtaagcttctaagcATGATCAAGgtgctcacacagtttatccacaac ctgagtggatgacatcaagataggtcgttgtatctccttcctctcgtactctcatgacca cggaaagATGATCAAGagaggatgatttcttggccatatcgcaatgaatacttgtgactt gtgcttccaattgacatcttcagcgccatattgcgctggccaaggtgacggagcgggatt acgaaagcatgatcatggctgttgttctgtttatcttgttttgactgagacttgttagga tagacggtttttcatcactgactagccaaagccttactctgcctgacatcgaccgtaaat tgataatgaatttacatgcttccgcgacgatttacctCTTGATCATcgatccgattgaag atcttcaattgttaattctcttgcctcgactcatagccatgatgagctCTTGATCATgtt tccttaaccctctattttttacggaagaATGATCAAGctgctgctCTTGATCATcgtttc`

Finding a 9-mer that appears six times (either as itself or as its reverse complement) in a DNA string of length 500 is far more surprising than finding a 9-mer that appears three times (as itself). This observation leads us to the working hypothesis that ATGATCAAG and its reverse complement CTTGATCAT indeed represent DnaA boxes in Vibrio cholerae. This computational conclusion makes sense biologically because the DnaA protein that binds to DnaA boxes and initiates replication does not care which of the two strands it binds to. Thus, for our purposes, both ATGATCAAG and CTTGATCAT represent DnaA boxes.

However, before concluding that we have found the DnaA box of Vibrio cholerae, the careful bioinformatician should check if there are other short regions in the Vibrio cholerae genome exhibiting multiple occurrences of ATGATCAAG (or CTTGATCAT). After all, maybe these strings occur as repeats throughout the entire Vibrio cholerae genome, rather than just in the ori region. To this end, we need to solve the following problem.

```
Pattern Matching Problem: Find all occurrences of a pattern in a string.
     Input: Strings Pattern and Genome.
     Output: All starting positions in Genome where Pattern appears as a substring.
```

In [8]:
def PatternMatching(Text, Pattern):
    Positions = []
    l = len(Pattern)
    for i in range((len(Text) + 1) - l):
        if Text[i:i+l] == Pattern:
            Positions.append(i)
    return Positions

In [9]:
genome, pattern = 'GATATATGCATATACTT', 'ATAT'

PatternMatching(genome, pattern)

[1, 3, 9]

If we remove the final symbol of Pattern, denoted LastSymbol(Pattern), then we will obtain a (k − 1)-mer that we denote as Prefix(Pattern). The observation from the previous step therefore generalizes to the following formula: 

PatternToNumber(Pattern) = 4 · PatternToNumber(Prefix(Pattern)) + SymbolToNumber(LastSymbol(Pattern))

This equation leads to the following recursive algorithm, i.e., a program that calls itself. If you want to learn more about recursive algorithms, see "DETOUR: The Towers of Hanoi" in the print companion.

```
   PatternToNumber(Pattern)
        if Pattern contains no symbols
            return 0
        symbol ← LastSymbol(Pattern)
        Prefix ← Prefix(Pattern)
        return 4 · PatternToNumber(Prefix) + SymbolToNumber(symbol)
```

In [10]:
GENOME_LETTERS = ['A', 'C', 'G', 'T']

def SymbolToNumber(Symbol):
    return GENOME_LETTERS.index(Symbol)

def NumberToSymbol(index):
    return GENOME_LETTERS[index]

In [11]:
def PatternToNumber(Pattern):
    if len(Pattern) == 0:
        return 0
    symbol = Pattern[-1]
    Prefix = Pattern[:-1]
    return 4 * PatternToNumber(Prefix) + SymbolToNumber(symbol)

In [12]:
PatternToNumber('AGT')

11

In [13]:
PatternToNumber('TTTCCACTCTTACTTAT')

16997282291

In the pseudocode below, we denote the quotient and the remainder when dividing integer n by integer m as Quotient(n, m) and Remainder(n, m), respectively. For example, Quotient(11, 4) = 2 and Remainder(11, 4) = 3. This pseudocode uses the function NumberToSymbol(index), which is the inverse of SymbolToNumber and transforms the integers 0, 1, 2, and 3 into the respective symbols A, C, G, and T.

```
NumberToPattern(index, k)
    if k = 1
        return NumberToSymbol(index)
    prefixIndex ← Quotient(index, 4)
    r ← Remainder(index, 4)
    symbol ← NumberToSymbol(r)
    PrefixPattern ← NumberToPattern(prefixIndex, k − 1)
    return concatenation of PrefixPattern with symbol
```

In [14]:
from math import floor

def NumberToPattern(index, k):
    if k == 1:
        return NumberToSymbol(index)
    prefixIndex = floor(index / 4)
    r = index % 4
    symbol = NumberToSymbol(r)
    PrefixPattern = NumberToPattern(prefixIndex, k - 1)
    return PrefixPattern + symbol

In [15]:
NumberToPattern(5631, 9)

'AACCCTTTT'

The pseudocode below generates a frequency array by first initializing every element in the frequency array to zero (4k operations) and then making a single pass down Text (approximately |Text| · k operations). For each k-mer Pattern that we encounter, we add 1 to the value of the frequency array corresponding to Pattern. As before, we refer to the k-mer beginning at position i of Text as Text(i, k).

```
ComputingFrequencies(Text, k)
    for i ← 0 to 4k − 1
        FrequencyArray(i) ← 0
    for i ← 0 to |Text| − k
        Pattern ← Text(i, k)
        j ← PatternToNumber(Pattern)
        FrequencyArray(j) ← FrequencyArray(j) + 1
    return FrequencyArray
```

In [16]:
def ComputingFrequency(Text, k):
    FrequencyArray = [0] * 4**k
    for i in range((len(Text) + 1) - k):
        Pattern = Text[i:i+k]
        j = PatternToNumber(Pattern)
        FrequencyArray[j] += 1
    return FrequencyArray

In [17]:
genome, k = 'ACGCGGCTCTGAAA', 2

ComputingFrequency(genome, k)

[2, 1, 0, 0, 0, 0, 2, 2, 1, 2, 1, 0, 0, 1, 1, 0]

We now have a faster algorithm for the Frequent Words Problem. After generating the frequency array, we can find all most frequent k-mers by simply finding all k-mers corresponding to the maximum element(s) in the frequency array.


```
FasterFrequentWords(Text, k)
    FrequentPatterns ← an empty set
    FrequencyArray ← ComputingFrequencies(Text, k)
    maxCount ← maximal value in FrequencyArray
    for i ← 0 to 4k − 1
        if FrequencyArray(i) = maxCount
            Pattern ← NumberToPattern(i, k)
            add Pattern to the set FrequentPatterns
    return FrequentPatterns
```

In [18]:
def FasterFrequentWords(Text, k):
    FrequentPatterns = []
    FrequencyArray = ComputingFrequency(Text, k)
    maxCount = max(FrequencyArray)
    for i in range(0, 4**k):
        if FrequencyArray[i] == maxCount:
            Pattern = NumberToPattern(i, k)
            FrequentPatterns.append(Pattern)
    return set(FrequentPatterns)

In [19]:
genome, k = 'ACGTTGCATGTCGCATGATGCATGAGAGCT', 4

FasterFrequentWords(genome, k)

{'CATG', 'GCAT'}

The pseudocode below slides a window of length L down Genome. After computing the frequency array for the current window, it identifies (L, t)-clumps simply by finding which k-mers occur at least t times within the window. To keep track of these clumps, our algorithm uses an array Clump of length 4k whose values are all initialized to zero. For each value of i between 0 and 4k − 1, we will set Clump(i) equal to 1 if NumberToPattern(i, k) forms an (L, t)-clump in Genome.

```
ClumpFinding(Genome, k, t, L)
    FrequentPatterns ← an empty set
    for i ← 0 to 4k − 1
        Clump(i) ← 0
    for i ← 0 to |Genome| − L
        Text ← the string of length L starting at position i in Genome 
        FrequencyArray ← ComputingFrequencies(Text, k)
        for index ← 0 to 4k − 1
            if FrequencyArray(index) ≥ t
                Clump(index) ← 1
    for i ← 0 to 4k − 1
        if Clump(i) = 1
            Pattern ← NumberToPattern(i, k)
            add Pattern to the set FrequentPatterns
    return FrequentPatterns
```

In [20]:
def ClumpFinding(Genome, k, t, L):
    FrequentPatterns = []
    Clump = [0] * 4**k
    for i in range(len(Genome) - L):
        Text = Genome[i:i+L]
        FrequencyArray = ComputingFrequency(Text, k)
        for ii in range(0, 4**k):
            if FrequencyArray[ii] >= t:
                Clump[ii] = 1
    for key, val in enumerate(Clump):
        if val == 1:
            Pattern = NumberToPattern(key, k)
            FrequentPatterns.append(Pattern)
    return set(FrequentPatterns)

In [21]:
genome = 'CGGACTCGACAGATGTGAAGAACGACAATGTGAAGACTCGACACGACAGAGTGAAGAGAAGAGGAAACATTGTAA'

ClumpFinding(genome, 5, 4, 50)

{'CGACA', 'GAAGA'}

To improve ClumpFinding, we observe that when we slide our window of length L one symbol to the right, the frequency array does not change much, and so regenerating the frequency array from scratch is inefficient. For example, the figure below shows the frequency arrays (k = 2) for the 13-mers Text = AAGCAAAGGTGGG and Text′ = AGCAAAGGTGGGC starting at positions 0 and 1 of the 14-mer AAGCAAAGGTGGGC. These two frequency arrays differ in only two elements corresponding to the first k-mer in Text (AA) and the last k-mer in Text’ (GC). Specifically, the frequency array value corresponding to the first k-mer of Text is reduced by 1 in the frequency array of Text’, and the frequency array value corresponding to the last k-mer of Text is increased by 1 in the frequency array of Text’.

<img src="http://bioinformaticsalgorithms.com/images/Replication/frequency_array_consecutive_strings.png">

The observation on the previous step helps us modify ClumpFinding as shown below. Note that we now only call ComputingFrequencies once, updating the frequency array as we go along.

```
BetterClumpFinding(Genome, k, t, L)
    FrequentPatterns ← an empty set
    for i ← 0 to 4k − 1
        Clump(i) ← 0
    Text ← Genome(0, L)
    FrequencyArray ← ComputingFrequencies(Text, k)
    for i ← 0 to 4k − 1
        if FrequencyArray(i) ≥ t
            Clump(i) ← 1
    for i ← 1 to |Genome| − L
        FirstPattern ← Genome(i − 1, k)
        index ← PatternToNumber(FirstPattern)
        FrequencyArray(index) ← FrequencyArray(index) − 1
        LastPattern ← Genome(i + L − k, k)
        index ← PatternToNumber(LastPattern)
        FrequencyArray(index) ← FrequencyArray(index) + 1
        if FrequencyArray(index) ≥ t
            Clump(index) ← 1
    for i ← 0 to 4k − 1
        if Clump(i) = 1
            Pattern ← NumberToPattern(i, k)
            add Pattern to the set FrequentPatterns
    return FrequentPatterns
```

In [22]:
def BetterClumpFinding(Genome, k, t, L):
    FrequentPatterns = []
    Clump = [0] * 4**k
    Text = Genome[:L]
    FrequencyArray = ComputingFrequency(Text, k)
    for key, _ in enumerate(Clump):
        if FrequencyArray[key] >= t:
            Clump[key] = 1
    for i in range(1, len(Genome) - L):
        d = i - 1
        FirstPattern = Genome[d:d+k]
        index = PatternToNumber(FirstPattern)
        FrequencyArray[index] -= 1
        
        d = i + L - k
        LastPattern = Genome[d:d+k]
        index = PatternToNumber(LastPattern)
        FrequencyArray[index] += 1
        
        if FrequencyArray[index] >= t:
            Clump[index] = 1
    for key, val in enumerate(Clump):
        if val == 1:
            Pattern = NumberToPattern(key, k)
            FrequentPatterns.append(Pattern)
    return set(FrequentPatterns)        

In [23]:
genome = 'CGGACTCGACAGATGTGAAGAACGACAATGTGAAGACTCGACACGACAGAGTGAAGAGAAGAGGAAACATTGTAA'

BetterClumpFinding(genome, 5, 4, 50)

{'CGACA', 'GAAGA'}

In [24]:
#  How many different 9-mers form 
# (500,3)-clumps in the E. coli genome? 
# In other words, do not count a 9-mer more than once.
with open('./E_coli.txt') as file:
    genome = file.readline().strip()
    res = BetterClumpFinding(genome, 9, 3, 500)
    print(len(res))

1904


### Deamination

In the last section, we saw that as the replication fork expands, DNA polymerase synthesizes DNA quickly on the reverse half-strand but suffers delays on the forward half-strand. We will explore the asymmetry of DNA replication to design a new algorithm for finding ori.

How in the world can the asymmetry of replication possibly help us locate ori? Notice that since the replication of a reverse half-strand proceeds quickly, it lives double-stranded for most of its life. Conversely, a forward half-strand spends a much larger amount of its life single-stranded, waiting to be used as a template for replication. This discrepancy between the forward and reverse half-strands is important because single-stranded DNA has a much higher mutation rate than double-stranded DNA. In particular, if one of the four nucleotides in single-stranded DNA has a greater tendency than other nucleotides to mutate in single-stranded DNA, then we should observe a shortage of this nucleotide on the forward half-strand.

Following up on this thought, let’s compare the nucleotide counts of the reverse and forward half-strands. If these counts differ substantially, then we will design an algorithm that attempts to track down these differences in genomes for which ori is unknown. The nucleotide counts for the forward and reverse half-strands of the Thermotoga petrophila genome are shown in the table below.

<img src="http://bioinformaticsalgorithms.com/images/Replication/forward_reverse_nucleotide_counts.png" width="50%">

Although the frequencies of A and T are practically identical on the two half-strands, C is more frequent on the reverse half-strand than on the forward half-strand, resulting in a difference of 219518 - 207901 = +11617. Its complementary nucleotide G is less frequent on the reverse half-strand than on the forward half-strand, resulting in a difference of 201634 - 211607 = -9973.

It turns out that we observe these discrepancies because cytosine (C) has a tendency to mutate into thymine (T) through a process called deamination. Deamination rates rise 100-fold when DNA is single-stranded, which leads to a decrease in cytosine on the forward half-strand. Also, since C-G base pairs eventually change into T-A base pairs, deamination results in the observed decrease in guanine (G) on the reverse half-strand (recall that a forward parent half-strand synthesizes a reverse daughter half-strand, and vice-versa).

### Skew Diagram

Let's see if we can take advantage of these peculiar statistics caused by deamination to locate ori. As the table on the previous step illustrates, the difference between the total amount of guanine and the total amount of cytosine is negative on the reverse half-strand (201634 - 219518 = -17884) and positive on the forward half-strand (211607 - 207901 = 3706). Thus, our idea is to traverse the genome, keeping a running total of the difference between the counts of G and C. If this difference starts increasing, then we guess that we are on the forward half-strand; on the other hand, if this difference starts decreasing, then the we guess that we are on the reverse half-strand. See the figure below.

<img src="http://bioinformaticsalgorithms.com/images/Replication/increasing_decreasing_skew.png"  width="50%">

Since we don't know the location of ori in a circular genome, let's linearize it (i.e., select an arbitrary position and pretend that the genome begins here), resulting in a linear string Genome. We define Skewi(Genome) as the difference between the total number of occurrences of G and the total number of occurrences of C in the first i nucleotides of Genome. The skew diagram is defined by plotting Skewi (Genome) (as i ranges from 0 to |Genome|), where Skew0 (Genome) is set equal to zero. The figure below shows a skew diagram for the DNA string CATGGGCATCGGCCATACGCC.

Note that we can compute Skewi+1(Genome) from Skewi(Genome) according to the nucleotide in position i of Genome. If this nucleotide is G, then Skewi+1(Genome) = Skewi(Genome) + 1; if this nucleotide is C, then Skewi+1(Genome)= Skewi(Genome) – 1; otherwise, Skewi+1(Genome) = Skewi(Genome).

<img src="http://bioinformaticsalgorithms.com/images/Replication/skew_diagram_basic.png" width="75%">

In [25]:
def Skew(Genome):
    return Genome.upper().count('G') - Genome.upper().count('C')

def SkewSequence(Genome):
    Positions = []
    for i in range(len(Genome)):
        Skew_i = Skew(Genome[:i])
        Positions.append(Skew_i)
    return Positions

In [26]:
genome = 'CATGGGCATCGGCCATACGCC'

SkewSequence(genome)

[0, -1, -1, -1, 0, 1, 2, 1, 1, 1, 0, 1, 2, 1, 0, 0, 0, 0, -1, 0, -1]

Let's follow the 5' → 3' direction of DNA and walk along the chromosome from ter to ori (along a reverse half-strand), then continue on from ori to ter (along a forward half-strand). In our previous discussion, we saw that the skew is decreasing along the reverse half-strand and increasing along the forward half-strand. Thus, the skew should achieve a minimum at the position where the reverse half-strand ends and the forward half-strand begins, which is exactly the location of ori!

We have just developed an insight for a new algorithm for locating ori: it should be found where the skew attains a minimum.

```
Minimum Skew Problem: Find a position in a genome where the skew diagram attains a minimum.
     Input: A DNA string Genome.
     Output: All integer(s) i minimizing Skewi (Genome) among all values of i (from 0 to |Genome|).
```

In [27]:
def MinimumSkew(Genome):
    Skews = SkewSequence(Genome)
    Positions = []
    Minimum = min(Skews)
    for pos, val in enumerate(Skews):
        if val == Minimum:
            Positions.append(pos)
    return Positions

In [28]:
genome = 'TAAAGACTGCCGAGAGGCCAACACGAGTGCTAGAACGAGGGGCGTAAACGCGGGTCCGAT'

MinimumSkew(genome)

[11, 24]

We say that position i in k-mers p1 … pk and q1 … qk is a mismatch if pi ≠ qi. For example, CGAAT and CGGAC have two mismatches. The number of mismatches between strings p and q is called the Hamming distance between these strings and is denoted HammingDistance(p, q).

```
Hamming Distance Problem: Compute the Hamming distance between two strings.
     Input: Two strings of equal length.
     Output: The Hamming distance between these strings.
```

In [29]:
def HammingDistance(Sequence, OtherSequence):
    if not len(Sequence) == len(OtherSequence):
        raise ArgumentError('Both sequences must have the same length.')
    Distance = 0
    for idx, gene in enumerate(Sequence):
        if not gene == OtherSequence[idx]:
            Distance += 1
    return Distance

In [30]:
a, b = 'GGGCCGTTGGT', 'GGACCGTTGAC'

HammingDistance(a, b)

3

Our goal is to generate the d-neighborhood Neighbors(Pattern, d), the set of all k-mers whose Hamming distance from Pattern does not exceed d. As a warm up, we will first generate the 1-neigborhood of Pattern using the following pseudocode.

```
ImmediateNeighbors(Pattern)
    Neighborhood ← the set consisting of single string Pattern
    for i = 1 to |Pattern|
        symbol ← i-th nucleotide of Pattern
        for each nucleotide x different from symbol
            Neighbor ← Pattern with the i-th nucleotide substituted by x
            add Neighbor to Neighborhood
    return Neighborhood
```

In [31]:
def ImmediateNeighbors(Pattern):
    Neighborhood = [Pattern]
    for idx, symbol in enumerate(Pattern):
        Letters = GENOME_LETTERS.copy()
        Letters.remove(symbol)
        for nucleotide in Letters:
            Neighbor = list(Pattern)
            Neighbor[idx] = nucleotide
            Neighborhood.append("".join(Neighbor))
    return set(Neighborhood)

In [32]:
genome = 'ACG'

ImmediateNeighbors(genome)

{'AAG', 'ACA', 'ACC', 'ACG', 'ACT', 'AGG', 'ATG', 'CCG', 'GCG', 'TCG'}

Now, consider a (k − 1)-mer Pattern’ belonging to Neighbors(Suffix(Pattern), d). By the definition of the d-neighborhood Neighbors(Suffix(Pattern), d), we know that HammingDistance(Pattern′, Suffix(Pattern)) is either equal to d or less than d. In the first case, we can add FirstSymbol(Pattern) to the beginning of Pattern’ in order to obtain a k-mer belonging to Neighbors(Pattern, d). In the second case, we can add any symbol to the beginning of Pattern’ and obtain a k-mer belonging to Neighbors(Pattern, d).

In the following pseudocode, we use the notation symbol • Text to denote the concatenation of a character symbol and a string Text, e.g., A•GCATG = AGCATG.

```
Neighbors(Pattern, d)
    if d = 0
        return {Pattern}
    if |Pattern| = 1 
        return {A, C, G, T}
    Neighborhood ← an empty set
    SuffixNeighbors ← Neighbors(Suffix(Pattern), d)
    for each string Text from SuffixNeighbors
        if HammingDistance(Suffix(Pattern), Text) < d
            for each nucleotide x
                add x • Text to Neighborhood
        else
            add FirstSymbol(Pattern) • Text to Neighborhood
    return Neighborhood
```

In [33]:
def Neighbors(Pattern, d):
    if d == 0:
        return [Pattern]
    if len(Pattern) == 1:
        return GENOME_LETTERS
    Neighborhood = []
    SuffixNeighbors = Neighbors(Pattern[1:], d)
    for neighbor in SuffixNeighbors:
        if HammingDistance(Pattern[1:], neighbor) < d:
            for nucleotide in GENOME_LETTERS:
                Neighborhood.append(nucleotide + neighbor)
        else:
            Neighborhood.append(Pattern[0] + neighbor)
    return set(Neighborhood)

In [34]:
genome, d = 'AAAGACGC', 3

neighbors = Neighbors(genome, 3)
len(neighbors)

1789

Computing Countd(Text, Pattern) simply requires us to compute the Hamming distance between Pattern and every k-mer substring of Text, which is achieved by the following pseudocode.

```
ApproximatePatternCount(Text, Pattern, d)
    count ← 0
    for i ← 0 to |Text| − |Pattern|
        Pattern′ ← Text(i , |Pattern|)
        if HammingDistance(Pattern, Pattern′) ≤ d
            count ← count + 1
    return count
```

In [35]:
def ApproximatePatternCount(Text, Pattern, d):
    count = 0
    l = len(Pattern)
    for i in range((len(Text)+1) - l):
        Segment = Text[i:i+l]
        if HammingDistance(Pattern, Segment) <= d:
            count += 1
    return count

In [36]:
genome, pattern, d = 'TTTAGAGCCTTCAGAGG', 'GAGG', 2

ApproximatePatternCount(genome, pattern, d)

4

A most frequent k-mer with up to d mismatches in Text is simply a string Pattern maximizing Countd(Text, Pattern) among all k-mers. Note that Pattern does not need to actually appear as a substring of Text; for example, as we already saw, AAAAA is the most frequent 5-mer with 1 mismatch in AACAAGCTGATAAACATTTAAAGAG, even though it does not appear exactly in this string. Keep this in mind while solving the following problem.

```
Frequent Words with Mismatches Problem: Find the most frequent k-mers with mismatches in a string.
     Input: A string Text as well as integers k and d. (You may assume k ≤ 12 and d ≤ 3.)
     Output: All most frequent k-mers with up to d mismatches in Text.
```

One way to solve the above problem is to generate all 4k k-mers Pattern, compute ApproximatePatternCount(Text, Pattern, d) for each k-mer Pattern, and then output k-mers with the maximum number of approximate occurrences. This is an inefficient approach in practice, since many of the 4k k-mers that this method analyzes should not be considered because neither they nor their mutated versions (with up to d mismatches) appear in Text.

The following pseudocode reduces the Frequent Words with Mismatches Problem to sorting. The following pseudocode reduces the Frequent Words with Mismatches Problem to sorting. It first generates all neighbors (with up to d mismatches) for all k-mers in Text and combines them all into an array NeighborhoodArray. Note that a k-mer Pattern appears Countd(Text, Pattern) times in this array. The only thing left is to sort this array, count how many times each k-mer appears in the sorted array, and then return the k-mers that occur the maximum number of times.

```
FrequentWordsWithMismatches(Text, k, d)
    FrequentPatterns ← an empty set
    Neighborhoods ← an empty list
    for i ← 0 to |Text| − k
        add Neighbors(Text(i, k), d) to Neighborhoods
    form an array NeighborhoodArray holding all strings in Neighborhoods
    for i ← 0 to |Neighborhoods| − 1
        Pattern ← NeighborhoodArray(i) 
        Index(i) ← PatternToNumber(Pattern)
        Count(i) ← 1
    SortedIndex ← Sort(Index)
    for i ← 0 to |Neighborhoods| − 2 
        if SortedIndex(i) = SortedIndex(i + 1)
            Count(i + 1) ← Count(i) + 1
   maxCount ← maximum value in array Count
   for i ← 0 to |Neighborhoods| − 1
       if Count(i) = maxCount
           Pattern ← NumberToPattern(SortedIndex(i), k)
           add Pattern to FrequentPatterns
   return FrequentPatterns 
```

In [37]:
def FrequentWordsWithMistaches(Text, k, d):
    FrequentPatterns = []
    Neighborhoods = []
    for i in range((len(Text)+1) - k):
        Neighborhoods.append(Neighbors(Text[i:i+k], d))
    NeighborhoodArray = []
    for neighborhood in Neighborhoods:
        NeighborhoodArray += neighborhood
    Index, Count = [], []
    for Pattern in NeighborhoodArray:
        Index.append(PatternToNumber(Pattern))
        Count.append(1)
    Index.sort()
    for i in range(len(NeighborhoodArray) - 2):
        if Index[i] == Index[i + 1]:
            Count[i + 1] = Count[i] + 1
    maxCount = max(Count)
    for i in range(len(NeighborhoodArray)):
        if Count[i] == maxCount:
            Pattern = NumberToPattern(Index[i], k)
            FrequentPatterns.append(Pattern)
    return set(FrequentPatterns)

In [38]:
genome, k, d = 'ACGTTGCATGTCGCATGATGCATGAGAGCT', 4, 1

FrequentWordsWithMistaches(genome, k, d)

{'ATGC', 'ATGT', 'GATG'}

We now redefine the Frequent Words Problem to account for both mismatches and reverse complements. Recall that Patternrc refers to the reverse complement of Pattern.

```
Frequent Words with Mismatches and Reverse Complements Problem: Find the most frequent k-mers (with mismatches and reverse complements) in a string.
      Input: A DNA string Text as well as integers k and d.
      Output: All k-mers Pattern maximizing the sum Countd(Text, Pattern)+ Countd(Text, Patternrc) over all possible k-mers.
```

In [39]:
# TODO

### Motifs

Given a collection of strings Dna and an integer d, a k-mer is a (k,d)-motif if it appears in every string from Dna with at most d mismatches. For example, the implanted 15-mer in the strings above represents a (15,4)-motif.

```
Implanted Motif Problem: Find all (k, d)-motifs in a collection of strings.
     Input: A collection of strings Dna, and integers k and d.
     Output: All (k, d)-motifs in Dna.
```

Brute force (also known as exhaustive search) is a general problem-solving technique that explores all possible solution candidates and checks whether each candidate solves the problem. Such algorithms require little effort to design and are guaranteed to produce a correct solution, but they may take an enormous amount of time, and the number of candidates may be too large to check.

A brute force approach for solving the Implanted Motif Problem is based on the observation that any (k, d)-motif must be at most d mismatches apart from some k-mer appearing in the first string in Dna. Therefore, we can generate all such k-mers and then check which of them are (k, d)-motifs. If you have forgotten how to generate these k-mers.

```
MotifEnumeration(Dna, k, d)
    Patterns ← an empty set
    for each k-mer Pattern in Dna
        for each k-mer Pattern’ differing from Pattern by at most d mismatches
            if Pattern' appears in each string from Dna with at most d ﻿mismatches
                add Pattern' to Patterns
    remove duplicates from Patterns
    return Patterns
```

In [40]:
def MotifEnumeration(Dna, k, d):
    Patterns = []
    Kmers = []
    l = len(Dna[0])
    for dna in Dna:
        for i in range(l - k):
            Kmers.append(dna[i:i+k])
    for kmer in set(Kmers):
        for neighbor in Neighbors(kmer, d):
            In = True
            for dna in Dna:
                if ApproximatePatternCount(dna, neighbor, d) == 0:
                    In = False
                    break
            if In:
                Patterns.append(neighbor)
    return set(Patterns)

In [41]:
dna = [
    'ATTTGGC',
    'TGCCTTA',
    'CGGTATC',
    'GAAAATT' ]

MotifEnumeration(dna, 3, 1)

{'ATA', 'ATT', 'GTT', 'TTT'}

By varying the choice of k-mers in each sequence, we can construct a large number of different motif matrices from a given sample of DNA sequences. Our goal is to select k-mers resulting in the most “conserved” motif matrix, meaning the matrix with the most upper case letters (and thus the fewest number of lower case letters). Leaving aside the question of how we select such k-mers, we will first focus on how to score the resulting motif matrices, defining Score(Motifs) as the number of unpopular (lower case) letters in the motif matrix Motifs. Our goal is to find a collection of k-mers that minimizes this score.

<img src="http://bioinformaticsalgorithms.com/images/Motifs/motifs_score.png" width="75%">

In [42]:
def ScoreMotifs(Motifs):
    Scores = []
    Count = []
    Length = len(Motifs)
    for Col in range(len(Motifs[0])):
        Count.append({'A':0,'C':0,'G':0,'T':0})
        for Row in range(Length):
            Count[Col][Motifs[Row][Col].upper()] += 1
    for Score in Count:
        PredominantLetter = max(Score, key=lambda i: Score[i])
        Total = sum(Score.values())
        Scores.append(Total - Score[PredominantLetter])
    return Scores

In [43]:
Motifs = [
    "TCGGGGGTTTTT",
    "CCGGTGACTTAC",
    "ACGGGGATTTTC",
    "TTGGGGACTTTT",
    "AAGGGGACTTCC",
    "TTGGGGACTTCC",
    "TCGGGGATTCAT",
    "TCGGGGATTCCT",
    "TAGGGGAACTAC",
    "TCGGGTATAACC"
]

scores = ScoreMotifs(Motifs)
print(" + ".join(map(str, scores)), "=", sum(scores))

3 + 4 + 0 + 0 + 1 + 1 + 1 + 5 + 2 + 3 + 6 + 4 = 30


We can construct the 4 × k count matrix Count(Motifs) counting the number of occurrences of each nucleotide in each column of the motif matrix; the (i, j)-th element of Count(Motifs) stores the number of times that nucleotide i appears in column j of Motifs. We will further divide all of the elements in the count matrix by t, the number of rows in Motifs. This results in a profile matrix P = Profile(Motifs) for which Pi,j is the frequency of the i-th nucleotide in the j-th column of the motif matrix. Note that the elements of any column of the profile matrix sum to 1. The figure below shows the motif, count, and profile matrices for the NF-κB binding sites.

<img src="http://bioinformaticsalgorithms.com/images/Motifs/motifs_score_count_profile.png" width="70%">

In [44]:
def CountMotifs(Motifs):
    Counts = []
    for Col in range(len(Motifs[0])):
        Count = {'A':0,'T':0,'C':0,'G':0}
        for Motif in Motifs:
            Count[Motif[Col]] += 1
        Counts.append(Count)
    return Counts

def ScoreMotifs(Motifs):
    Scores = []
    Counts = CountMotifs(Motifs)
    for Count in Counts:
        PredominantLetter = max(Count, key=lambda i: Count[i])
        Total = sum(Count.values())
        Scores.append(Total - Count[PredominantLetter])
    return sum(Scores)

def ProfileMotifs(Motifs):
    Profiles = []
    Counts = CountMotifs(Motifs)
    for Count in Counts:
        Profile = {}
        Total = sum(Count.values())
        for Letter in GENOME_LETTERS:
            Profile[Letter] = Count[Letter] / Total
        Profiles.append(Profile)
    return Profiles

In [45]:
CountMotifs(Motifs)

[{'A': 2, 'C': 1, 'G': 0, 'T': 7},
 {'A': 2, 'C': 6, 'G': 0, 'T': 2},
 {'A': 0, 'C': 0, 'G': 10, 'T': 0},
 {'A': 0, 'C': 0, 'G': 10, 'T': 0},
 {'A': 0, 'C': 0, 'G': 9, 'T': 1},
 {'A': 0, 'C': 0, 'G': 9, 'T': 1},
 {'A': 9, 'C': 0, 'G': 1, 'T': 0},
 {'A': 1, 'C': 4, 'G': 0, 'T': 5},
 {'A': 1, 'C': 1, 'G': 0, 'T': 8},
 {'A': 1, 'C': 2, 'G': 0, 'T': 7},
 {'A': 3, 'C': 4, 'G': 0, 'T': 3},
 {'A': 0, 'C': 6, 'G': 0, 'T': 4}]

In [46]:
ProfileMotifs(Motifs)

[{'A': 0.2, 'C': 0.1, 'G': 0.0, 'T': 0.7},
 {'A': 0.2, 'C': 0.6, 'G': 0.0, 'T': 0.2},
 {'A': 0.0, 'C': 0.0, 'G': 1.0, 'T': 0.0},
 {'A': 0.0, 'C': 0.0, 'G': 1.0, 'T': 0.0},
 {'A': 0.0, 'C': 0.0, 'G': 0.9, 'T': 0.1},
 {'A': 0.0, 'C': 0.0, 'G': 0.9, 'T': 0.1},
 {'A': 0.9, 'C': 0.0, 'G': 0.1, 'T': 0.0},
 {'A': 0.1, 'C': 0.4, 'G': 0.0, 'T': 0.5},
 {'A': 0.1, 'C': 0.1, 'G': 0.0, 'T': 0.8},
 {'A': 0.1, 'C': 0.2, 'G': 0.0, 'T': 0.7},
 {'A': 0.3, 'C': 0.4, 'G': 0.0, 'T': 0.3},
 {'A': 0.0, 'C': 0.6, 'G': 0.0, 'T': 0.4}]

In [47]:
ScoreMotifs(Motifs)

30

Finally, we form a consensus string, denoted Consensus(Motifs), from the most popular letters in each column of the motif matrix. If we select Motifs correctly from the collection of upstream regions, then Consensus(Motifs) provides an ideal candidate regulatory motif for these regions. For example, the consensus string for the NF-κB binding sites in the figure below is TCGGGGATTTCC.

<img src="http://bioinformaticsalgorithms.com/images/Motifs/motifs_score_count_profile_consensus.png" width="70%">

In [48]:
import random

def ConsensusMotif(Motifs):
    Motif = []
    Profiles = ProfileMotifs(Motifs)
    for Profile in Profiles:
        PredominantLetter = max(Profile, key=lambda i: Profile[i])
        Motif.append(PredominantLetter)
    return Motif

In [49]:
ConsensusMotif(Motifs)

['T', 'C', 'G', 'G', 'G', 'G', 'A', 'T', 'T', 'T', 'C', 'C']

Every column of Profile(Motifs) corresponds to a probability distribution, or a collection of nonnegative numbers that sum to 1. For example, the 2nd column in the profile matrix for the NF-κB binding sites corresponds to the probabilities 0.2, 0.6, 0.0, and 0.2 for A, C, G, and T, respectively.

<img src="http://bioinformaticsalgorithms.com/images/Motifs/profile_matrix.png" width="50%">

Entropy is a measure of the uncertainty of a probability distribution (p1, …, pN), and is defined as follows:

H(p1,…,pN)=−∑i=1Npi⋅log2pi

For example, the entropy of the probability distribution (0.2, 0.6, 0.0, 0.2) corresponding to the 2nd column of the NF-κB profile matrix is

−(0.2log20.2+0.6log20.6+0.0log20.0+0.2log20.2)≈1.371

Note: Technically, log2(0) is undefined, but in the computation of entropy, we assume that 0 · log2(0) is equal to 0.

In [50]:
from math import log2

def MotifsEntropy(Motifs):
    Entropies = []
    Profiles = ProfileMotifs(Motifs)
    for Profile in Profiles:
        Entropy = sum(map(lambda x: -x*log2(x) if x != 0 else 0, Profile.values()))
        Entropies.append(Entropy)
    return Entropies

In [51]:
entropy = MotifsEntropy(Motifs)

print(entropy, sum(entropy))

[1.1567796494470395, 1.3709505944546687, 0.0, 0.0, 0.4689955935892812, 0.4689955935892812, 0.4689955935892812, 1.360964047443681, 0.9219280948873623, 1.1567796494470395, 1.5709505944546684, 0.9709505944546686] 9.916290005356972


The first potential issue with implementing MedianString is writing a function to compute d(Pattern, Dna) = ∑ti=1 d(Pattern, Dnai), the sum of distances between Pattern and each string in Dna = {Dna1, ..., Dnat}. This task is achieved by the following pseudocode.

```
DistanceBetweenPatternAndStrings(Pattern, Dna)
    k ← |Pattern|
    distance ← 0
    for each string Text in Dna
        HammingDistance ← ∞
        for each k-mer Pattern’ in Text
            if HammingDistance > HammingDistance(Pattern, Pattern’)
                HammingDistance ← HammingDistance(Pattern, Pattern’)
        distance ← distance + HammingDistance
    return distance
```

In [52]:
from math import inf

def DistanceBetweenPatternAndStrings(Pattern, Dna):
    k = len(Pattern)
    distance = 0
    for Text in Dna:
        hammingDistance = inf
        for i in range((len(Text)+1) - k):
            kmer = Text[i:i+k]
            d = HammingDistance(Pattern, kmer)
            if hammingDistance > d:
                hammingDistance = d
        distance += hammingDistance
    return distance

In [53]:
pattern = 'AAA'
dna = 'TTACCTTAAC GATATCTGTC ACGGCGTTCG CCCTAAAGAG CGTCAGAGGT'.split(" ")

DistanceBetweenPatternAndStrings(pattern, dna)

5

To solve the Median String Problem, we need to iterate through all possible 4k k-mers Pattern before computing d(Pattern, Dna). The pseudocode below is a modification of MedianString using the function NumberToPattern (implemented in Charging Station: Converting Patterns Into Numbers and Vice-Versa), which is applied to convert all integers from 0 to 4k - 1 into all possible k-mers.

```
MedianString(Dna, k)
    distance ← ∞
    for i ←0 to 4k −1
        Pattern ← NumberToPattern(i, k)
        if distance > DistanceBetweenPatternAndStrings(Pattern, Dna)
            distance ← DistanceBetweenPatternAndStrings(Pattern, Dna)
            Median ← Pattern
    return Median
```

In [54]:
def MedianString(Dna, k):
    Median = None
    distance = inf
    for i in range(4**k):
        Pattern = NumberToPattern(i, k)
        d = DistanceBetweenPatternAndStrings(Pattern, Dna)
        if distance > d:
            distance = d
            Median = Pattern
    return Median

In [55]:
dna = "AAATTGACGCAT GACGACCACGTT CGTCAGCGCCTG GCTGAGCACCGG AGTTCGGGACAG".split(" ")
k = 3

MedianString(dna, 3)

'GAC'

Many algorithms are iterative procedures that must choose among many alternatives at each iteration. Some of these alternatives may lead to correct solutions, whereas others may not. Greedy algorithms select the “most attractive” alternative at each iteration. For example, a greedy algorithm in chess might attempt to capture an opponent’s most valuable piece at every move. Yet anyone who has played chess knows that this strategy of looking only one move ahead will likely produce disastrous results. In general, most greedy algorithms typically fail to find an exact solution of the problem; instead, they are often fast heuristics that trade accuracy for speed in order to find an approximate solution. Nevertheless, for many biological problems that we will study in this book, greedy algorithms will prove quite useful.

In this section, we will explore a greedy approach to motif finding. Again, let Motifs be a collection of k-mers taken from t strings Dna. Recall from our discussion of entropy that we can view each column of Profile(Motifs) as a four-sided biased die. Thus, a profile matrix with k columns can be viewed as a collection of k dice, which we will roll to randomly generate a k-mer. For example, if the first column of the profile matrix is (0.2, 0.1, 0.0, 0.7), then we generate A as the first nucleotide with probability 0.2, C with probability 0.1, G with probability 0.0, and T with probability 0.7.

Below, we reproduce the profile matrix for the NF-κB binding sites, where the lone colored entry in the i-th column corresponds to the i-th nucleotide in ACGGGGATTACC. The probability Pr(ACGGGGATTACC | Profile) that Profile generates ACGGGGATTACC is computed by simply multiplying the highlighted entries in the profile matrix.

<img src="http://bioinformaticsalgorithms.com/images/Motifs/probability_random_string.png" width="70%">

In [56]:
from functools import reduce

def ProbabilityMotif(Pattern, Profile):
    Probabilities = []
    for Index, Letter in enumerate(Pattern):
        Probability = Profile[Index][Letter]
        Probabilities.append(Probability)
    return reduce(lambda x, y: x*y, Probabilities, 1)

In [57]:
pattern = 'ACGGGGATTACC'

ProbabilityMotif(pattern, ProfileMotifs(Motifs))

0.0008398080000000002

In [58]:
pattern = 'TCGTGGATTTCC'

ProbabilityMotif(pattern, ProfileMotifs(Motifs))

0.0

Given a profile matrix Profile, we can evaluate the probability of every k-mer in a string Text and find a Profile-most probable k-mer in Text, i.e., a k-mer that was most likely to have been generated by Profile among all k-mers in Text. For example, ACGGGGATTACC is the Profile-most probable 12-mer in GGTACGGGGATTACCT. Indeed, every other 12-mer in this string has probability 0. In general, if there are multiple Profile-most probable k-mers in Text, then we select the first such k-mer occurring in Text.

```
Profile-most Probable k-mer Problem: Find a Profile-most probable k-mer in a string.
     Input: A string Text, an integer k, and a 4 × k matrix Profile.
     Output: A Profile-most probable k-mer in Text.
```

In [59]:
def MostProbableKmer(Text, k, Profile):
    Kmers = []
    MostProbable = Text[0:k]
    for i in range((len(Text)+1) - k):
        Kmers.append(Text[i:i+k])
    Probability = 0
    for kmer in Kmers:
        p = ProbabilityMotif(kmer, Profile)
        if p > Probability:
            MostProbable = kmer
            Probability = p
    return MostProbable

In [60]:
text = 'ACCTGTTTATTGCCTAAGTTCCGAACAAACCCAATATAGCCCGAGGGCCT'
k = 5
profile = [
    {'A':0.2,'C':0.4,'G':0.3,'T':0.1},
    {'A':0.2,'C':0.3,'G':0.3,'T':0.2},
    {'A':0.3,'C':0.1,'G':0.5,'T':0.1},
    {'A':0.2,'C':0.5,'G':0.2,'T':0.1},
    {'A':0.3,'C':0.1,'G':0.4,'T':0.2}
]

MostProbableKmer(text, k, profile)

'CCGAG'

In [61]:
#with open('./dataset_159_3.txt') as file:
#    text = file.readline().strip()
#    k = int(file.readline().strip())
#    lines = file.readlines()
#    profile = [{} for x in lines[0].split(" ")]
#    for row, line in enumerate(lines):
#        for col, p in enumerate(line.strip().split(" ")):
#            profile[col][GENOME_LETTERS[row]] = float(p)
#    print(MostProbableKmer(text, k, profile))

Our proposed greedy motif search algorithm, GreedyMotifSearch, starts by forming a motif matrix from arbitrarily selected k-mers in each string from Dna (which in our specific implementation is the first k﻿-mer in each string). It then attempts to improve this initial motif matrix by trying each of the k-mers in Dna1 as the first motif. For a given choice of k-mer Motif1 in Dna1, it builds a profile matrix Profile for this lone k-mer, and sets Motif2 equal to the Profile-most probable k-mer in Dna2. It then iterates by updating Profile as the profile matrix formed from Motif1 and Motif2, and sets Motif3 equal to the Profile-most probable k-mer in Dna3. In general, after finding i − 1 k-mers Motifs in the first i − 1 strings of Dna, GreedyMotifSearch constructs Profile(Motifs) and selects the Profile-most probable k-mer from Dnai based on this profile matrix. After obtaining a k-mer from each string to obtain a collection Motifs, GreedyMotifSearch tests to see whether Motifs outscores the current best scoring collection of motifs and then moves Motif1 one symbol over in Dna1, beginning the entire process of generating Motifs again.

```
GreedyMotifSearch(Dna, k, t)
    BestMotifs ← motif matrix formed by first k-mers in each string from Dna
    for each k-mer Motif in the first string from Dna
        Motif1 ← Motif
        for i = 2 to t
            form Profile from motifs Motif1, …, Motifi - 1
            Motifi ← Profile-most probable k-mer in the i-th string in Dna
        Motifs ← (Motif1, …, Motift)
        if Score(Motifs) < Score(BestMotifs)
            BestMotifs ← Motifs
    return BestMotifs
```

In [62]:
def GreedyMotifSearch(Dna, k):
    BestMotifs = [Text[0:k] for Text in Dna]
    l = len(Dna[0]) + 1
    for i in range(l - k):
        Motifs = [Dna[0][i:i+k]]
        for ii in range(1, len(Dna)):
            Profile = ProfileMotifs(Motifs)
            Motifs.append(MostProbableKmer(Dna[ii], k, Profile))
        if ScoreMotifs(Motifs) < ScoreMotifs(BestMotifs):
            BestMotifs = Motifs
    return BestMotifs

In [63]:
k, t = 3, 5
dna = "GGCGTTCAGGCA AAGAATCAGTCA CAAGGAGTTCGC CACGTCAATCAC CAATAATATTCG".split(" ")

GreedyMotifSearch(dna, k)

['CAG', 'CAG', 'CAA', 'CAA', 'CAA']

In [64]:
#with open('./dataset_159_5.txt') as file:
#    meta = file.readline().strip().split(" ")
#    dna = list(map(str.strip, file.readlines()))
#    motifs = GreedyMotifSearch(dna, int(meta[0]))
#    for motif in motifs:
#        print(motif)

Cromwell’s rule is relevant to the calculation of the probability of a string based on a profile matrix. For example, consider the following Profile.

<img src="http://bioinformaticsalgorithms.com/images/Motifs/greedy_profile_1.png" width="70%">

The fourth symbol of TCGTGGATTTCC causes Pr(TCGTGGATTTCC, Profile) to be equal to zero. As a result, the entire string is assigned a zero probability, even though TCGTGGATTTCC differs from the consensus string at only one position. For that matter, TCGTGGATTTCC has the same low probability as AAATCTTGGAA, which is very different from the consensus string.

In order to improve this unfair scoring, bioinformaticians often substitute zeroes with small numbers called pseudocounts. The simplest approach to introducing pseudocounts, called Laplace’s Rule of Succession, is similar to the principle that Laplace used to calculate the probability that the sun will not rise tomorrow. In the case of motifs, pseudocounts often amount to adding 1 (or some other small number) to each element of Count(Motifs). For example, say we have the following motif, count, and profile matrices:

<img src="http://bioinformaticsalgorithms.com/images/Motifs/greedy_profile_2.png" width="70%">

Laplace’s Rule of Succession adds 1 to each element of Count(﻿Motifs), updating the two matrices to the following:

<img src="http://bioinformaticsalgorithms.com/images/Motifs/greedy_profile_3.png" width="70%">


In [127]:
import random

def LaplaceCountMotifs(Motifs):
    Counts = []
    for Col in range(len(Motifs[0]) ):
        Count = {'A':1,'T':1,'C':1,'G':1}
        for Motif in Motifs:
            Count[Motif[Col]] += 1
        Counts.append(Count)
    return Counts

def LaplaceProfileMotifs(Motifs):
    Profiles = []
    Counts = LaplaceCountMotifs(Motifs)
    for Count in Counts:
        Profile = {}
        Total = sum(Count.values())
        for Letter in GENOME_LETTERS:
            Profile[Letter] = int((Count[Letter] / Total) * 100) + random.randint(0,1)
        Profiles.append(Profile)
    return Profiles

After incorporating Laplace's Rule of Succession, GreedyMotifSearch is given by the following:

```
GreedyMotifSearch(Dna, k, t)
    form a set of k-mers BestMotifs by selecting 1st k-mers in each string from Dna
    for each k-mer Motif in the first string from Dna
        Motif1 ← Motif
        for i = 2 to t
            apply Laplace's Rule of Succession to form Profile from motifs Motif1, …, Motifi-1
            Motifi ← Profile-most probable k-mer in the i-th string in Dna
        Motifs ← (Motif1, …, Motift)
        if Score(Motifs) < Score(BestMotifs)
            BestMotifs ← Motifs
    output BestMotifs
```

In [66]:
def LaplaceGreedyMotifSearch(Dna, k):
    BestMotifs = [Text[0:k] for Text in Dna]
    l = len(Dna[0]) + 1
    for i in range(l - k):
        Motifs = [Dna[0][i:i+k]]
        for ii in range(1, len(Dna)):
            Profile = LaplaceProfileMotifs(Motifs)
            Motifs.append(MostProbableKmer(Dna[ii], k, Profile))
        if ScoreMotifs(Motifs) < ScoreMotifs(BestMotifs):
            BestMotifs = Motifs
    return BestMotifs

In [67]:
k = 3
dna = "GGCGTTCAGGCA AAGAATCAGTCA CAAGGAGTTCGC CACGTCAATCAC CAATAATATTCG".split(" ")

GreedyMotifSearch(dna, k), LaplaceGreedyMotifSearch(dna, k)

(['CAG', 'CAG', 'CAA', 'CAA', 'CAA'], ['TTC', 'ATC', 'TTC', 'ATC', 'TTC'])

We previously defined Profile(Motifs) as the profile matrix constructed from a collection of k-mers Motifs in Dna. Now, given a collection of strings Dna and an arbitrary 4 x k matrix Profile, we define Motifs(Profile, Dna) as the collection of k-mers formed by the Profile-most probable k-mers in each string from Dna. For example, consider the following Profile and Dna: 

<img src="http://bioinformaticsalgorithms.com/images/Motifs/randomized_profile_1.png" width="30%">

Taking the Profile-most probable 4-mer from each row of Dna produces the following 4-mers (shown in red):

<img src="http://bioinformaticsalgorithms.com/images/Motifs/randomized_profile_2.png" width="30%">

In general, we can begin from a collection of randomly chosen k-mers Motifs in Dna, construct Profile(Motifs), and use this profile to generate a new collection of k-mers:

  Motifs(Profile(Motifs), Dna).

Why would we do this? Because our hope is that Motifs(Profile(Motifs), Dna) has a better score than the original collection of k-mers Motifs. We can then form the profile matrix of these k-mers,

Profile(Motifs(Profile(Motifs), Dna))

and use it to form the most probable k-mers,

Motifs(Profile(Motifs(Profile(Motifs), Dna)), Dna).

We can continue to iterate. . .

...Profile(Motifs(Profile(Motifs(Profile(Motifs), Dna)), Dna))...

for as long as the score of the constructed motifs keeps improving, which is exactly what RandomizedMotifSearch does. To implement this algorithm, you will need to randomly select the initial collection of k-mers that form the motif matrix Motifs. To do so, you will need a random number generator (denoted Random(N)) that is equally likely to return any integer from 1 to N. You might like to think about this random number generator as an unbiased N-sided die.

```
RandomizedMotifSearch(Dna, k, t)
    randomly select k-mers Motifs = (Motif1, …, Motift) in each string from Dna
    BestMotifs ← Motifs
    while forever
        Profile ← Profile(Motifs)
        Motifs ← Motifs(Profile, Dna)
        if Score(Motifs) < Score(BestMotifs)
            BestMotifs ← Motifs
        else
            return BestMotifs
```


In [115]:
def ScoreMotifs(Motifs):
    Score = 0
    Consensus = ConsensusMotif(Motifs)
    for Motif in Motifs:
        Score += HammingDistance(Motif, Consensus)
    return Score

def MotifsFromProfile(Profile, Dna):
    Motifs = []
    k = len(Profile)
    for dna in Dna:
        Kmer = MostProbableKmer(dna, k, Profile)
        Motifs.append(Kmer)
    return Motifs

In [131]:
import random

random.seed(0)

def RandomizedMotifSearch(Dna, k):
    Motifs = []
    for dna in Dna:
        t = random.randint(0, len(dna)-k)
        Motifs.append(dna[t:t+k])
    BestMotifs = Motifs
    while True:
        Profile = LaplaceProfileMotifs(Motifs)
        Motifs = MotifsFromProfile(Profile, Dna)
        if ScoreMotifs(Motifs) < ScoreMotifs(BestMotifs):
            BestMotifs = Motifs
        else:
            return BestMotifs

In [130]:
dna = [
    "CGCCCCTCTCGGGGGTGTTCAGTAAACGGCCA",
    "GGGCGAGGTATGTGTAAGTGCCAAGGTGCCAG",
    "TAGTACCGAGACCGAAAGAAGTATACAGGCGT",
    "TAGATCAAGTTTCAGGTGCACGTCGGTGAACC",
    "AATCCACCAGCTCCACGTGCAATGTTGGCCTA"]

BestMotifs = RandomizedMotifSearch(dna, 8)

for i in range(1000):
    Motifs = RandomizedMotifSearch(dna, 8)
    if ScoreMotifs(Motifs) < ScoreMotifs(BestMotifs):
        BestMotifs = Motifs

BestMotifs

['TCTCGGGG', 'CCAAGGTG', 'TACAGGCG', 'TTCAGGTG', 'TCCACGTG']

Note that RandomizedMotifSearch may change all t strings Motifs in a single iteration. This strategy may prove reckless, since some correct motifs (captured in Motifs) may potentially be discarded at the next iteration. GibbsSampler is a more cautious iterative algorithm that discards a single k-mer from the current set of motifs at each iteration and decides to either keep it or replace it with a new one. This algorithm thus moves with more caution in the space of all motifs, as illustrated below.

<img src="http://bioinformaticsalgorithms.com/images/Motifs/randomized_vs_gibbs.png" width="70%">

Like RandomizedMotifSearch, GibbsSampler starts with randomly chosen k-mers in each of t DNA sequences, but it makes a random rather than a deterministic choice at each iteration. It uses randomly selected k-mers (Motif1, …, Motift) to come up with another (hopefully better scoring) set of k-mers. In contrast with RandomizedMotifSearch (which deterministically defines new motifs as

Motifs(Profile(Motifs), Dna),

GibbsSampler randomly selects an integer i between 1 and t and then randomly changes only a single k-mer Motifi.

To describe how GibbsSampler updates Motifs, we will need a slightly more advanced random number generator. Given a probability distribution (p1, …, pn), this random number generator, denoted Random(p1, …, pn), models an n-sided biased die and returns integer i with probability pi. For example, the standard six-sided fair die represents the random number generator Random(1/6, 1/6, 1/6, 1/6, 1/6, 1/6), whereas a biased die might represent the random number generator Random(0.1, 0.2, 0.3, 0.05, 0.1, 0.25). GibbsSampler further generalizes the random number generator by using the function Random(p1, …, pn) defined for any set of non-negative numbers, i.e., not necessarily satisfying the condition that the pi sum to 1. If the pi sum to some C > 0 instead, then Random(p1, …, pn) is defined as Random(p1/C, …, pn/C), where (p1/C, …, pn/C) is a probability distribution. For example, for (0.1, 0.2, 0.3) with 0.1 + 0.2 + 0.3 = 0.6,

Random(0.1, 0.2, 0.3) = Random(0.1/0.6, 0.2/0.6, 0.3/0.6) = Random(1/6, 1/3, 1/2).

We have previously defined the notion of a Profile-most probable k-mer in a string. We now define a Profile-randomly generated k-mer in a string Text. For each k-mer Pattern in Text, compute the probability Pr(Pattern | Profile), resulting in n = |Text| - k + 1 probabilities (p1, …, pn). These probabilities do not necessarily sum to 1, but we can still form the random number generator Random(p1, …, pn) based on them. GibbsSampler uses this random number generator to select a Profile-randomly generated k-mer at each step: if the die rolls the number i, then we define the Profile-randomly generated k-mer as the i-th k-mer in Text. While the pseudocode below repeats this procedure N times, in practice GibbsSampler depends on various stopping rules that are beyond the scope of this chapter.

```
GibbsSampler(Dna, k, t, N)
    randomly select k-mers Motifs = (Motif1, …, Motift) in each string from Dna
    BestMotifs ← Motifs
    for j ← 1 to N
        i ← Random(t)
        Profile ← profile matrix constructed from all strings in Motifs except for Motifi
        Motifi ← Profile-randomly generated k-mer in the i-th sequence
        if Score(Motifs) < Score(BestMotifs)
            BestMotifs ← Motifs
    return BestMotifs
```

In [142]:
import numpy as np

def GibbsSampler(Dna, k, N):
    Motifs = []
    for dna in Dna:
        t = np.random.randint(0, len(dna)-k)
        Motifs.append(dna[t:t+k])
    t = len(Dna) - 1
    BestMotifs = Motifs
    for _ in range(N):
        i = np.random.randint(0, t)
        Profile = LaplaceProfileMotifs(Motifs)
        Motifs[i] = MotifsFromProfile(Profile, Dna)[i]
        if ScoreMotifs(Motifs) < ScoreMotifs(BestMotifs):
            BestMotifs = Motifs
    return BestMotifs

In [143]:
dna = [
    "CGCCCCTCTCGGGGGTGTTCAGTAAACGGCCA",
    "GGGCGAGGTATGTGTAAGTGCCAAGGTGCCAG",
    "TAGTACCGAGACCGAAAGAAGTATACAGGCGT",
    "TAGATCAAGTTTCAGGTGCACGTCGGTGAACC",
    "AATCCACCAGCTCCACGTGCAATGTTGGCCTA"]

BestMotifs = GibbsSampler(dna, 8, 100)

for i in range(100):
    Motifs = GibbsSampler(dna, 8, 100)
    if ScoreMotifs(Motifs) < ScoreMotifs(BestMotifs):
        BestMotifs = Motifs

BestMotifs

['GGGGTGTT', 'GAGGTATG', 'GAAGTATA', 'CAGGTGCA', 'CACGTGCA']

In [144]:
dna = [
    "TGACGTTC",
    "TAAGAGTT",
    "GGACGAAA",
    "CTGTTCGC"]

first_motif = [
    "TGA",
    "GTT",
    "GAA",
    "TGT"]

MotifsFromProfile(ProfileMotifs(first_motif),dna)


['TGA', 'TAA', 'GGA', 'TGT']

In [155]:
a = """0   4   3   4   0   0  18   0   1   6  10   7   0   3   0   0   2   1   9   3
5   3   4   0   0   0   0  19   9   0   3   8   0   4  19  17  16   6   1   2
5   3  11  15  16  19   0   0   2   2   0   0  18   7   0   0   1   0   6  10
9   9   1   0   3   0   1   0   7  11   6   4   1   5   0   2   0  12   3   4""".split("\n")

In [156]:
a = list(map(lambda x: x.split("   "), a))

In [158]:
print(a)

[['0', '4', '3', '4', '0', '0  18', '0', '1', '6  10', '7', '0', '3', '0', '0', '2', '1', '9', '3'], ['5', '3', '4', '0', '0', '0', '0  19', '9', '0', '3', '8', '0', '4  19  17  16', '6', '1', '2'], ['5', '3  11  15  16  19', '0', '0', '2', '2', '0', '0  18', '7', '0', '0', '1', '0', '6  10'], ['9', '9', '1', '0', '3', '0', '1', '0', '7  11', '6', '4', '1', '5', '0', '2', '0  12', '3', '4']]
